<a href="https://colab.research.google.com/github/gwegayhu/dashboards-app/blob/master/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade transformers

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install --upgrade transformers
# %%
!pip install -U langchain-community
# %%
!pip install gdown
!pip install faiss-cpu

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline # Import HuggingFacePipeline
from transformers import pipeline # Import pipeline for creating HuggingFacePipeline

import gdown

# Download the Google Doc using gdown
url = "https://docs.google.com/document/d/15eOe5gup8mE0FiE5JUEYjreb94BIFUA-pvsApDrwPkQ/edit?usp=sharing"
output = "downloaded_document.txt"  # Choose a filename for the downloaded document
gdown.download(url, output, quiet=False)

# Load the document
with open(output, "r") as f:  # Open the downloaded file
    document = f.read()

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Create a HuggingFacePipeline instance
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
)
llm = HuggingFacePipeline(pipeline=pipe) # Create a HuggingFacePipeline object using the pipeline

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create a vector store
vectorstore = FAISS.from_texts([document], embeddings)

# Create a retriever
retriever = vectorstore.as_retriever()

# Create a RAG chain
# Pass the HuggingFacePipeline object (llm) to RetrievalQA.from_llm
rag_chain = RetrievalQA.from_llm(
    llm, # Pass the llm object here
    retriever=retriever,
)

# Ask a question
query = "What is the main topic of this document?"

# Generate an answer
result = rag_chain.run(query)

print(result)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=15eOe5gup8mE0FiE5JUEYjreb94BIFUA-pvsApDrwPkQ
  warnings.warn(
Downloading...
From: https://docs.google.com/document/d/15eOe5gup8mE0FiE5JUEYjreb94BIFUA-pvsApDrwPkQ/edit?usp=sharing
To: /content/downloaded_document.txt
219kB [00:00, 1.51MB/s]
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (139268 > 512). Running this sequence through the model will result in indexing errors
